In [1]:
import sys
sys.path.append('../code')
from resnet import *
# from funcs import *
from cifar_very_tiny import *
from cifar_tiny import *
from cifar_dataset import *    
import torch as t 
import numpy as np
from numpy import polyfit
from numpy import polyval
import tqdm
import matplotlib.pylab as plt
import matplotlib.cm as cm
import json
# import hyperparams
from importlib import reload
from scipy.interpolate import interp1d
from PIL import Image
%matplotlib inline
plt.rcParams['figure.figsize']=(12,9)
plt.rcParams['font.size']= 20

In [2]:
epoch_num = 25
# epoch_num = 50

run_num = 2 # количество запусков эксперимента

# версия нужна, чтобы различать старые и новые результаты экспериментов. 
# менять нужно каждый раз, когда есть хотя бы незначительные изменения в эксперименте
experiment_version = '3'

validate_every_epoch = 5 

# train_splines_every_epoch = 5 # каждые 5 эпох отслеживать траекторию гиперпараметров
# train_splines_every_epoch = 2
# train_splines_every_epoch = 3
train_splines_every_epoch = 10

# размер мини-эпохи в батчах, за которую у нас производится либо обучение спайлов, либо их использование
mini_epoch_size = 10

start_beta = 0.5
start_temp  = 1.0

In [3]:
train_loader_no_augumentation, valid_loader, test_loader = cifar10_loader(batch_size=128, split_train_val=True,
                                                                             maxsize=128*100, use_aug=False)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../data/cifar-10-python.tar.gz to ../data
Files already downloaded and verified


In [4]:
device = 'cuda' if t.cuda.is_available() else 'cpu'

In [79]:
from importlib import reload
import hyperparams
reload(hyperparams)
AdamHyperGradCalculator = hyperparams.AdamHyperGradCalculator

In [77]:
class Student(Cifar_Very_Tiny):
    def __init__(self):
        Cifar_Very_Tiny.__init__(self, 10)
        self.mu_feat = nn.Linear(128, 64).cuda()
        self.log_sigma_feat = torch.nn.Parameter(torch.zeros(1).cuda())
        self.bad_params = set(self.mu_feat.parameters())
        self.bad_params.add(self.log_sigma_feat)
        
    def inference_parameters(self):
        
        for p in self.parameters():
            if p not in self.bad_params:
                yield p 

In [87]:
# определяем функцию потерь как замкнутую относительно аргументов функцию
# нужно для подсчета градиентов гиперпараметров по двухуровневой оптимизации
def param_loss_mi(batch,model,h):
    lam1 = h[0]    
    x,y,teacher_feat, teacher_logit = batch        
    student_feat, student_logits = model.get_features(x, [3,4])
    #print (teacher_feat.shape, teacher_logit.shape, student_feat.shape, student_logits.shape)
    class_loss = crit(student_logits, y)
    sigma2 = torch.log(1+torch.exp(model.log_sigma_feat))
    feat_loss = ((model.mu_feat(teacher_feat) - student_feat)**2).sum(1).mean()/(2*sigma2) + 0.5*torch.log(sigma2)*np.prod(teacher_feat.shape[1:])
    loss = class_loss * (1.0-lam1) + feat_loss * lam1 * 1e-4
    return loss

# определяем функцию валидационную функцию потерь как замкнутую относительно аргументов функцию
# нужно для подсчета градиентов гиперпараметров по двухуровневой оптимизации
def hyperparam_loss_mi(batch, model):
    x,y = batch
    student_feat, student_logits = model.get_features(x, [3,4])
    class_loss = crit(student_logits, y)             
    return class_loss 

crit = nn.CrossEntropyLoss()

def dist_with_opt(experiment_version, train_loader_no_augumentation, test_loader, validation_loader, validate_every_epoch, lambdas=None, clip_grad=10e-3, seed=42):
    np.random.seed(seed)
    t.manual_seed(seed)
    
    # for lam1 in [1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 0.1, 0.5]:
    lam1 = t.nn.Parameter(t.tensor(np.random.uniform(low=0.0, high=1.0), device=device), requires_grad=True)
    
    if lambdas is not None: # non-random initialization
        lam1.data *= 0
        lam1.data += lambdas[0]

    student = Student().cuda()
    teacher = Cifar_Tiny(10).cuda() 
    teacher.load_state_dict(torch.load('../tiny_cifar10.model', map_location=torch.device(device)))
    #scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size=10, gamma=0.5)   
    

    #mu_logit = nn.Linear(10, 10).cpu()
    #log_sigma_logit = torch.nn.Parameter(torch.zeros(1).cpu())
    h =[lam1]
    print (lam1)
    optim = torch.optim.SGD(list(student.parameters()), lr=1.0)    
    scheduler = t.optim.lr_scheduler.StepLR(optim, step_size=10, gamma=0.5)
    optim2 = torch.optim.Adam(h, lr=1e-3)   
    #net, parameters_loss_function, hyperparameters_loss_function, optimizer, h
    hyper_grad_calc = AdamHyperGradCalculator(student, param_loss_mi, hyperparam_loss_mi, optim, h)
    val_load_iter = iter(validation_loader)
    internal_results = []
    for e in range(25):
        tq = tqdm.tqdm(train_loader_no_augumentation)
        losses = []

        for batch_id, (x,y) in enumerate(tq):
            try:
                 (v_x, v_y) = next(val_load_iter)
            except:                    
                val_load_iter = iter(validation_loader)
                (v_x, v_y) = next(val_load_iter)

                
            x = x.to(device)
            y = y.to(device)

            v_x = v_x.to(device)
            v_y = v_y.to(device)
            optim2.zero_grad()
            teacher_feat, teacher_logits = teacher.get_features(x, [3,4])
            
            hyper_grad_calc.calc_gradients((x,y, teacher_feat, teacher_logits), (v_x, v_y))                    
            t.nn.utils.clip_grad_value_(h, clip_grad)

            for h_ in h:
                if h_.grad is not None:
                    h_.grad = t.where(t.isnan(h_.grad), t.zeros_like(h_.grad), h_.grad)
            
            optim2.step()
            if lam1 > 1.0:
                lam1.data*=0.0
                lam1.data+=1.0
            if lam1 < 0.0:
                lam1.data*=0.0
                   
            optim.zero_grad()
            student_feat, student_logits = student.get_features(x, [3,4])
            # class_loss = crit(student_logits, y)
            # sigma2 = torch.log(1+torch.exp(log_sigma_feat))
            # feat_loss = ((mu_feat(teacher_feat) - student_feat)**2).sum(1).mean()/(2*sigma2) + 0.5*torch.log(sigma2)*np.prod(teacher_feat.shape[1:])
            #logit_loss =((mu_feat(teacher_feat) - student_feat)**2).sum(1).mean()/(2*sigma2) + 0.5*torch.log(sigma2)*np.prod(teacher_feat.shape[1:])
            #lam1 = 0.5
            loss = param_loss_mi((x,y, teacher_feat, teacher_logits), student,h)
            loss.backward()
            optim.step()
            losses.append(loss.detach().cpu().numpy())
            tq.set_description('current loss:{}'.format(np.mean(losses[-10:])))      
        scheduler.step()
        # student.eval()

        if e==0 or (e+1)%validate_every_epoch == 0:
            test_loss = []
            student.eval()
            for x,y in test_loader:
                x = x.to(device)
                y = y.to(device)
                student_feat, student_logits = student.get_features(x, [2,3])
                test_loss.append(crit(student_logits, y).detach().cpu().numpy())
            test_loss = float(np.mean(test_loss))
            val_loss = []
            for x,y in validation_loader:
                x = x.to(device)
                y = y.to(device)
                student_feat, student_logits = student.get_features(x, [2,3])
                val_loss.append(crit(student_logits, y).detach().cpu().numpy())
            val_loss = float(np.mean(val_loss))
        
        ac = float(accuracy(student, test_loader))
        student.train()

        # if not hyperopt:
        internal_results.append({'epoch': e, 'test loss':test_loss, 'val loss':val_loss, 'accuracy':ac,
                             'lambda1':float(lam1.cpu().detach().numpy()),
                            })
        # else:
        #     val_acc = float(accuracy(student, validation_loader))
        #     internal_results.append({'epoch': e, 'test loss':test_loss, 'val loss':val_loss, 'accuracy':ac,
        #                          'lambda1':float(lam1.cpu().detach().numpy()),
        #                           'val acc':val_acc})
        print (internal_results[-1])

    # if not hyperopt: # outer function optimization
    with open('../logs/acc_mi_'+experiment_version+'.txt','a') as out:
        out.write(json.dumps({'results':internal_results, 'version': exp_ver})+'\n')
    # else:
    #     # inner function for hyperopt optimization
    #     return max([res['val acc'] for res in internal_results])
        
        # with open('../logs/acc_mi_'+experiment_version+'.txt','a') as out:
        #     out.write('{}: {}: {}\n'.format(lam1, e, ac))
dist_with_opt(experiment_version, train_loader_no_augumentation, test_loader, valid_loader, validate_every_epoch, lambdas=[1e-4])        

Parameter containing:
tensor(1.0000e-04, device='cuda:0', requires_grad=True)


current loss:1.6417592763900757: 100%|██████████| 90/90 [00:05<00:00, 15.25it/s]


{'epoch': 0, 'test loss': 4.4714860916137695, 'val loss': 4.489005088806152, 'accuracy': 0.33239999413490295, 'lambda1': 0.07375209033489227}


current loss:1.2948156595230103: 100%|██████████| 90/90 [00:05<00:00, 15.67it/s]


{'epoch': 1, 'test loss': 4.4714860916137695, 'val loss': 4.489005088806152, 'accuracy': 0.39499998092651367, 'lambda1': 0.16014908254146576}


...

current loss:0.0041466220282018185: 100%|██████████| 90/90 [00:05<00:00, 15.81it/s] 


{'epoch': 23, 'test loss': 4.308137893676758, 'val loss': 4.320822715759277, 'accuracy': 0.5809999704360962, 'lambda1': 0.9846453070640564}


current loss:0.004894235171377659: 100%|██████████| 90/90 [00:05<00:00, 15.66it/s] 


{'epoch': 24, 'test loss': 4.2877984046936035, 'val loss': 4.294524192810059, 'accuracy': 0.5809000134468079, 'lambda1': 0.9836641550064087}


NameError: name 'exp_ver' is not defined

In [21]:
def accuracy(student, t_load):
    student.eval()
    total = 0 
    correct = 0
    with t.no_grad():
        for x,y in t_load:
            x = x.to(device)
            y = y.to(device)
            out = student(x)
            correct += t.eq(t.argmax(out, 1), y).sum()
            total+=len(x)
    student.train()
    return (correct/total).cpu().detach().numpy()


In [ ]:
def dist_with_no_opt(experiment_version, train_loader_no_augumentation, test_loader, validation_loader, validate_every_epoch, lambdas=None, file=True, no_tqdm=False, clip_grad=10e-3, seed=42):
    np.random.seed(seed)
    t.manual_seed(seed)

    lam1 = t.nn.Parameter(t.tensor(np.random.uniform(low=0.0, high=1.0), device=device), requires_grad=True)

    if lambdas is not None: # non-random initialization
        lam1.data *= 0
        lam1.data += lambdas[0]
    
    student = Cifar_Very_Tiny(10).cpu()
    teacher = Cifar_Tiny(10).cpu() 
    teacher.load_state_dict(torch.load('tiny_cifar10.model?raw=true', map_location=torch.device('cpu')))
    #scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size=10, gamma=0.5)   
    mu_feat = nn.Linear(128, 64).cpu()
    log_sigma_feat = torch.nn.Parameter(torch.zeros(1).cpu())

    #mu_logit = nn.Linear(10, 10).cpu()
    #log_sigma_logit = torch.nn.Parameter(torch.zeros(1).cpu())


    optim = torch.optim.Adam(list(student.parameters()) + list(mu_feat.parameters()) + [log_sigma_feat])    
    val_load_iter = iter(val_load)

    for e in range(25):
        tq = tqdm.tqdm(train_loader_no_augumentation)
        if no_tqdm:
            tq = train_loader_no_augumentation
        losses = []

        for batch_id, (x,y) in enumerate(tq):
            try:
                 (v_x, v_y) = next(val_load_iter)
            except:                    
                val_load_iter = iter(val_load)
                (v_x, v_y) = next(val_load_iter)
                
            x = x.to(device)
            y = y.to(device)
            optim.zero_grad()
            student_feat, student_logits = student.get_features(x, [3,4])
            # class_loss = crit(student_logits, y)
            # sigma2 = torch.log(1+torch.exp(log_sigma_feat))
            # feat_loss = ((mu_feat(teacher_feat) - student_feat)**2).sum(1).mean()/(2*sigma2) + 0.5*torch.log(sigma2)*np.prod(teacher_feat.shape[1:])
            #logit_loss =((mu_feat(teacher_feat) - student_feat)**2).sum(1).mean()/(2*sigma2) + 0.5*torch.log(sigma2)*np.prod(teacher_feat.shape[1:])
            #lam1 = 0.5
            loss = param_loss_mi((x,y,teacher_logits,mu_feat,log_sigma_feat), student,lam1)

            loss.backward()
            optim.step()
            losses.append(loss.detach().cpu().numpy())
            if not no_tqdm:
                tq.set_description('current loss:{}'.format(np.mean(losses[-10:])))
        #scheduler.step()
        
        if e==0 or (e+1)%validate_every_epoch == 0:
            test_loss = []
            student.eval()
            for x,y in test_loader:
                x = x.to(device)
                y = y.to(device)
                student_feat, student_logits = student.get_features(x, [3,4])
                test_loss.append(crit(student_logits, y).detach().cpu().numpy())
            test_loss = float(np.mean(test_loss))
            val_loss = []
            for x,y in validation_loader:
                x = x.to(device)
                y = y.to(device)
                student_feat, student_logits = student.get_features(x, [3,4])
                val_loss.append(crit(student_logits, y).detach().cpu().numpy())
            val_loss = float(np.mean(val_loss))
        
        ac = float(accuracy(student, test_loader))
        
        if file:
            internal_results.append({'epoch': e, 'test loss':test_loss, 'val loss':val_loss, 'accuracy':ac,
                                 'lambda1':float(lam1.cpu().detach().numpy()),
                                })
        else:
            val_acc = float(accuracy(student, validation_loader))
            internal_results.append({'epoch': e, 'test loss':test_loss, 'val loss':val_loss, 'accuracy':ac,
                                 'lambda1':float(lam1.cpu().detach().numpy()),
                                  'val acc':val_acc})
        
        print (internal_results[-1])

    if file: # outer function optimization
        with open('../logs/acc_mi_'+experiment_version+'.txt','a') as out:
            out.write(json.dumps({'results':internal_results, 'version': experiment_version})+'\n')
    else:
        # inner function for hyperopt optimization
        return max([res['val acc'] for res in internal_results])
        
        # with open('../logs/acc_mi_'+experiment_version+'.txt','a') as out:
        #     out.write('{}: {}: {}\n'.format(lam1, e, ac))


def dist_hyperopt(experiment_version, run_num, tr_load, t_load, val_load, validate_every_epoch, trial_num):
    np.random.seed(42)
    t.manual_seed(42)

    for _ in range(run_num):
        cost_function = lambda lambdas: -dist_with_no_opt(experiment_version, train_loader_no_augumentation, test_loader, validation_loader, validate_every_epoch, lambdas = best_lambdas['lambda1'], file=False, no_tqdm=True) # validation accuracy * (-1) -> min
       
        best_lambdas = fmin(fn=cost_function,                             
        #space= [ hp.uniform('lambda1', 0.0, 1.0), hp.uniform('lambda2', 0.0, 1.0), hp.uniform('temp', 0.1, 10.0)],
        space= [ hp.uniform('lambda1', 0.0, 1.0)],  
        algo=tpe.suggest,
        max_evals=trial_num)
        #cifar_with_validation_set(exp_ver, 1, epoch_num, filename, tr_s_epoch, m_e, tr_load, t_load, val_load, validate_every_epoch, lambdas = [best_lambdas['lambda1'], best_lambdas['lambda2'], best_lambdas['temp']],  mode='no-opt')
        dist_with_no_opt(experiment_version, train_loader_no_augumentation, test_loader, validation_loader, validate_every_epoch, lambdas = best_lambdas['lambda1'])